In [2]:
import pandas as pd
import pandasql as ps
from datetime import datetime
import pymysql
import teradatasql
import os
import logging

# QUESTION 1

In [3]:
cust_tbl = pd.read_csv(r"C:\Users\knguy14\Desktop\Take-home-assignment\data\cust_tbl.csv")
acct_tbl = pd.read_csv(r"C:\Users\knguy14\Desktop\Take-home-assignment\data\acct_tbl.csv")
xref_tbl = pd.read_csv(r"C:\Users\knguy14\Desktop\Take-home-assignment\data\xref_tbl.csv")

In [4]:
print(cust_tbl)
print(acct_tbl)
print(xref_tbl)

  customer_id  customer_age
0           A            19
1           B            44
2           C            32
3           D            70
   account_id account_open_date
0           1         30-Oct-05
1           2          5-Apr-90
2           3            May-89
3           4        Jan 2 2020
4           5         17-Jul-99
   account_id customer_id
0           1           A
1           2           A
2           4           A
3           3           B
4           3           C
5           4           D
6           5           D


In [9]:
# Convert the account_open_date to a standard datetime format
acct_tbl['account_open_date'] = pd.to_datetime(acct_tbl['account_open_date'], errors='coerce')
print(acct_tbl)

   account_id account_open_date
0           1        2005-10-30
1           2        1990-04-05
2           3        1989-05-01
3           4        2020-01-02
4           5        1999-07-17


In [6]:
# Problem 1: Write a query that gets the earliest account_open_date, along with the account_id, for each customer_id.
task1 = '''
SELECT
    c.customer_id,
    x.account_id,
    DATE(MIN(a.account_open_date)) AS account_open_date
FROM
    cust_tbl c
JOIN
    xref_tbl x ON c.customer_id = x.customer_id
JOIN
    acct_tbl a ON x.account_id = a.account_id
GROUP BY
    c.customer_id;

'''
ps.sqldf(task1, locals())

,customer_id,account_id,account_open_date
0,A,2,1990-04-05
1,B,3,1989-05-01
2,C,3,1989-05-01
3,D,5,1999-07-17


# Question 2

In [11]:
# Connection parameters (hidden due to privacy)
host = "tdprod1cop1.ngco.com"                #Add Host
username = "knguy14"                         #Add Username
password = "TrangBia123"                     #Add Password

Note: As DATE is a reserved keyword in SQL, i replaced it with transaction_date 

In [12]:

# Establish connection to Teradata
with teradatasql.connect(host=host, user=username, password=password, logmech='LDAP') as conn:
    cursor = conn.cursor()
    
    # Step 1: Create the volatile table for Products
    create_products_table_query = """
    CREATE MULTISET VOLATILE TABLE Products (
        id INTEGER NOT NULL PRIMARY KEY,
        name VARCHAR(100),
        Price FLOAT
    ) ON COMMIT PRESERVE ROWS;
    """
    
    cursor.execute(create_products_table_query)
    
     # Step 2: Insert data into Products
    Products = [
        (1, 'Laptop', 999.99),
        (2, 'Smartphone', 599.99),
        (3, 'Tablet', 399.99),
        (4, 'Headphones', 199.99),
        (5, 'Smartwatch', 299.99)
    ]

    insert_products_query = "INSERT INTO Products (id, name, Price) VALUES (?, ?, ?)"
    cursor.executemany(insert_products_query, Products)
    print(Products)
    
    # Step 3: Create the volatile table for Transactions
    create_transactions_table_query = """
    CREATE MULTISET VOLATILE TABLE Transactions (
        id INTEGER NOT NULL PRIMARY KEY,
        user_id INTEGER,
        transaction_date DATE,
        product_id INTEGER,
        quantity INTEGER
    ) ON COMMIT PRESERVE ROWS;
    """
    
    cursor.execute(create_transactions_table_query)
    
    # Step 4: Insert sample Transaction data
    Transactions = [
    (1, 101, '2024-01-15', 1, 1),
    (2, 102, '2024-01-20', 2, 2),
    (3, 103, '2024-02-05', 3, 1),
    (4, 104, '2024-02-15', 4, 3),
    (5, 105, '2024-03-10', 5, 1),
    (6, 106, '2024-03-25', 1, 2),
    (7, 107, '2024-04-05', 2, 1),
    (8, 108, '2024-04-15', 3, 2),
    (9, 109, '2024-05-10', 4, 1),
    (10, 110, '2024-05-20', 5, 3)
    ]
    
    insert_transactions_query = """
    INSERT INTO Transactions (id, user_id, transaction_date, product_id, quantity) 
    VALUES (?, ?, CAST(? AS DATE), ?, ?)
    """
    cursor.executemany(insert_transactions_query, Transactions)
    print(Transactions)
    
    
    # Query to find the number of distinct transactions, users, and total sales for every month in 2024
    select_query = """
    SELECT 
        EXTRACT(YEAR FROM t.transaction_date) AS "Year",
        EXTRACT(MONTH FROM t.transaction_date) AS "Month",
        COUNT(DISTINCT t.id) AS Transactions,
        COUNT(DISTINCT t.user_id) AS Customers,
        SUM(p.Price * t.quantity) AS Sales
    FROM 
        Transactions t
    JOIN 
        Products p ON t.product_id = p.id
    WHERE 
        EXTRACT(YEAR FROM t.transaction_date) = 2024
    GROUP BY 
        EXTRACT(YEAR FROM t.transaction_date),
        EXTRACT(MONTH FROM t.transaction_date)
    ORDER BY 
        "Year", "Month";
    """

    # Execute the query and read the results into a DataFrame
    df = pd.read_sql(select_query, conn)

    # Display the DataFrame
    print(df)


[(1, 'Laptop', 999.99), (2, 'Smartphone', 599.99), (3, 'Tablet', 399.99), (4, 'Headphones', 199.99), (5, 'Smartwatch', 299.99)]
[(1, 101, '2024-01-15', 1, 1), (2, 102, '2024-01-20', 2, 2), (3, 103, '2024-02-05', 3, 1), (4, 104, '2024-02-15', 4, 3), (5, 105, '2024-03-10', 5, 1), (6, 106, '2024-03-25', 1, 2), (7, 107, '2024-04-05', 2, 1), (8, 108, '2024-04-15', 3, 2), (9, 109, '2024-05-10', 4, 1), (10, 110, '2024-05-20', 5, 3)]


C:\Users\knguy14\AppData\Local\Temp\ipykernel_105144\2735413796.py:86: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(select_query, conn)


   Year  Month  Transactions  Customers    Sales
0  2024      1             2          2  2199.97
1  2024      2             2          2   999.96
2  2024      3             2          2  2299.97
3  2024      4             2          2  1399.97
4  2024      5             2          2  1099.96


In [13]:
print(df)

   Year  Month  Transactions  Customers    Sales
0  2024      1             2          2  2199.97
1  2024      2             2          2   999.96
2  2024      3             2          2  2299.97
3  2024      4             2          2  1399.97
4  2024      5             2          2  1099.96
